<a href="https://colab.research.google.com/github/ChandanM3686/AI-Desktop-Assistant/blob/main/loromodel_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ERROR: Operation cancelled by user


In [3]:
import os
from PIL import Image
from torchvision import transforms

def preprocess_images(input_dir, output_dir, image_size=256):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    image_list = os.listdir(input_dir)

    preprocess = transforms.Compose([
        transforms.Resize((image_size, image_size)),
        transforms.ToTensor(),
    ])

    for image_name in image_list:
        image_path = os.path.join(input_dir, image_name)
        img = Image.open(image_path)
        img = preprocess(img)

        # Convert tensor back to PIL image for saving
        img = transforms.ToPILImage()(img)

        save_path = os.path.join(output_dir, image_name)
        img.save(save_path)

# Example usage:
input_dir = "/content/sample_data/images.zip"
output_dir = "preprocessed_images"
preprocess_images(input_dir, output_dir)


In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class LoRAModel(nn.Module):
    def __init__(self, input_channels=3, hidden_channels=64, num_layers=5, kernel_size=3):
        super(LoRAModel, self).__init__()

        self.layers = nn.ModuleList()

        # Input layer
        self.layers.append(nn.Conv2d(input_channels, hidden_channels, kernel_size, padding=kernel_size//2))

        # Hidden layers
        for _ in range(num_layers - 2):
            self.layers.append(nn.Conv2d(hidden_channels, hidden_channels, kernel_size, padding=kernel_size//2))

        # Output layer
        self.layers.append(nn.Conv2d(hidden_channels, input_channels, kernel_size, padding=kernel_size//2))

    def forward(self, x):
        for layer in self.layers[:-1]:
            x = F.relu(layer(x))
        x = self.layers[-1](x)
        return x


In [5]:
# Training hyperparameters
batch_size = 16
num_epochs = 20
learning_rate = 0.001

# Other hyperparameters
image_size = 256  # Assuming square images
input_channels = 3  # RGB images
hidden_channels = 64
num_layers = 5
kernel_size = 3


In [10]:
import os
from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torch.optim import Adam
from tqdm import tqdm


class LoRAModel(nn.Module):
    def __init__(self, input_channels=3, hidden_channels=64, num_layers=5, kernel_size=3):
        super(LoRAModel, self).__init__()

        self.layers = nn.ModuleList()


        self.layers.append(nn.Conv2d(input_channels, hidden_channels, kernel_size, padding=kernel_size//2))


        for _ in range(num_layers - 2):
            self.layers.append(nn.Conv2d(hidden_channels, hidden_channels, kernel_size, padding=kernel_size//2))


        self.layers.append(nn.Conv2d(hidden_channels, input_channels, kernel_size, padding=kernel_size//2))

    def forward(self, x):
        for layer in self.layers[:-1]:
            x = F.relu(layer(x))
        x = self.layers[-1](x)
        return x



class CustomImageDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_list = os.listdir(root_dir)

    def __len__(self):
        return len(self.image_list)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.image_list[idx])
        image = Image.open(img_name)
        if self.transform:
            image = self.transform(image)
        return image


eval_dataset = CustomImageDataset(root_dir="preprocessed_images", transform=transform)
eval_dataloader = DataLoader(eval_dataset, batch_size=1, shuffle=False)

batch_size = 16
num_epochs = 20
learning_rate = 0.001
image_size = 256
input_channels = 3
hidden_channels = 64
num_layers = 5
kernel_size = 3


def preprocess_images(input_dir, output_dir, image_size=256):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    image_list = os.listdir(input_dir)

    preprocess = transforms.Compose([
        transforms.Resize((image_size, image_size)),
        transforms.ToTensor(),
    ])

    for image_name in image_list:
        image_path = os.path.join(input_dir, image_name)
        img = Image.open(image_path)
        img = preprocess(img)


        img = transforms.ToPILImage()(img)

        save_path = os.path.join(output_dir, image_name)
        img.save(save_path)

model = LoRAModel(input_channels=input_channels, hidden_channels=hidden_channels, num_layers=num_layers, kernel_size=kernel_size)


criterion = nn.MSELoss()


optimizer = Adam(model.parameters(), lr=learning_rate)


input_dir = "/content/sample_data/images.zip"
output_dir = "preprocessed_images"
preprocess_images(input_dir, output_dir, image_size=image_size)


transform = transforms.Compose([
    transforms.ToTensor(),
])

dataset = CustomImageDataset(root_dir=output_dir, transform=transform)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(num_epochs):
    total_loss = 0
    for batch in tqdm(dataloader, desc=f"Epoch {epoch+1}/{num_epochs}"):

        images = batch.to(device)


        outputs = model(images)

        loss = criterion(outputs, images)


        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss / len(dataloader)}")


torch.save(model.state_dict(), "lora_model.pth")


Epoch 1/20: 100%|██████████| 1/1 [00:00<00:00,  4.55it/s]


Epoch 1/20, Loss: 0.34017181396484375


Epoch 2/20: 100%|██████████| 1/1 [00:00<00:00,  5.92it/s]


Epoch 2/20, Loss: 0.2736615836620331


Epoch 3/20: 100%|██████████| 1/1 [00:00<00:00,  5.82it/s]


...

Epoch 18/20, Loss: 0.009414376690983772


Epoch 19/20: 100%|██████████| 1/1 [00:00<00:00,  5.58it/s]


Epoch 19/20, Loss: 0.010952791199088097


Epoch 20/20: 100%|██████████| 1/1 [00:00<00:00,  5.79it/s]

Epoch 20/20, Loss: 0.014657577499747276


In [13]:
import os
from PIL import Image
import torch
import matplotlib.pyplot as plt
from torchvision.utils import save_image
from skimage.metrics import peak_signal_noise_ratio as PSNR
from skimage.metrics import structural_similarity as SSIM
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from torchvision.datasets import ImageFolder
from torchvision import transforms


class CustomImageDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_list = os.listdir(root_dir)

    def __len__(self):
        return len(self.image_list)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.image_list[idx])
        image = Image.open(img_name)
        if self.transform:
            image = self.transform(image)
        return image

model = LoRAModel(input_channels=input_channels, hidden_channels=hidden_channels, num_layers=num_layers, kernel_size=kernel_size)
model.load_state_dict(torch.load("lora_model.pth"))
model.eval()


def evaluate_model(model, dataloader, device):
    psnr_values = []
    ssim_values = []
    for images in tqdm(dataloader, desc="Evaluating"):
        images = images.to(device)
        with torch.no_grad():
            outputs = model(images)
        outputs = outputs.clamp(0, 1)
        for i in range(len(images)):
            original_img = images[i].permute(1, 2, 0).cpu().numpy()
            generated_img = outputs[i].permute(1, 2, 0).cpu().numpy()
            psnr = PSNR(original_img, generated_img)
            ssim = SSIM(original_img, generated_img, multichannel=True)
            psnr_values.append(psnr)
            ssim_values.append(ssim)
    avg_psnr = sum(psnr_values) / len(psnr_values)
    avg_ssim = sum(ssim_values) / len(ssim_values)
    return avg_psnr, avg_ssim

transform = transforms.Compose([
    transforms.ToTensor(),
])
eval_dataset = CustomImageDataset(root_dir="preprocessed_images", transform=transform)
eval_dataloader = DataLoader(eval_dataset, batch_size=1, shuffle=False)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
avg_psnr, avg_ssim = evaluate_model(model, eval_dataloader, device)
print(f"Average PSNR: {avg_psnr}, Average SSIM: {avg_ssim}")


Evaluating:   0%|          | 0/11 [00:00<?, ?it/s]<ipython-input-13-a68b61005b5c>:48: FutureWarning: `multichannel` is a deprecated argument name for `structural_similarity`. It will be removed in version 1.0. Please use `channel_axis` instead.
  ssim = SSIM(original_img, generated_img, multichannel=True)
Evaluating: 100%|██████████| 11/11 [00:00<00:00, 42.11it/s]

Average PSNR: 17.96419098630483, Average SSIM: 0.8217360594055869


In [14]:
import os
from PIL import Image
import torch
import matplotlib.pyplot as plt
from torchvision.utils import save_image
from skimage.metrics import peak_signal_noise_ratio as PSNR
from skimage.metrics import structural_similarity as SSIM
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from torchvision.datasets import ImageFolder
from torchvision import transforms
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
from tqdm import tqdm


class LoRAModel(nn.Module):
    def __init__(self, input_channels=3, hidden_channels=64, num_layers=5, kernel_size=3):
        super(LoRAModel, self).__init__()

        self.layers = nn.ModuleList()

        self.layers.append(nn.Conv2d(input_channels, hidden_channels, kernel_size, padding=kernel_size//2))

        for _ in range(num_layers - 2):
            self.layers.append(nn.Conv2d(hidden_channels, hidden_channels, kernel_size, padding=kernel_size//2))

        self.layers.append(nn.Conv2d(hidden_channels, input_channels, kernel_size, padding=kernel_size//2))

    def forward(self, x):
        for layer in self.layers[:-1]:
            x = F.relu(layer(x))
        x = self.layers[-1](x)
        return x


class CustomImageDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_list = os.listdir(root_dir)

    def __len__(self):
        return len(self.image_list)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.image_list[idx])
        image = Image.open(img_name)
        if self.transform:
            image = self.transform(image)
        return image


def preprocess_images(input_dir, output_dir, image_size=256):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    image_list = os.listdir(input_dir)

    preprocess = transforms.Compose([
        transforms.Resize((image_size, image_size)),
        transforms.ToTensor(),
    ])

    for image_name in image_list:
        image_path = os.path.join(input_dir, image_name)
        img = Image.open(image_path)
        img = preprocess(img)
        img = transforms.ToPILImage()(img)
        save_path = os.path.join(output_dir, image_name)
        img.save(save_path)


def evaluate_model(model, dataloader, device):
    psnr_values = []
    ssim_values = []
    for images in tqdm(dataloader, desc="Evaluating"):
        images = images.to(device)
        with torch.no_grad():
            outputs = model(images)
        outputs = outputs.clamp(0, 1)
        for i in range(len(images)):
            original_img = images[i].permute(1, 2, 0).cpu().numpy()
            generated_img = outputs[i].permute(1, 2, 0).cpu().numpy()
            psnr = PSNR(original_img, generated_img)
            ssim = SSIM(original_img, generated_img, multichannel=True)
            psnr_values.append(psnr)
            ssim_values.append(ssim)
    avg_psnr = sum(psnr_values) / len(psnr_values)
    avg_ssim = sum(ssim_values) / len(ssim_values)
    return avg_psnr, avg_ssim


batch_size = 16
num_epochs = 20
learning_rate = 0.001
image_size = 256
input_channels = 3
hidden_channels = 64
num_layers = 5
kernel_size = 3

input_dir = "/content/sample_data/images.zip"
output_dir = "preprocessed_images"
preprocess_images(input_dir, output_dir, image_size=image_size)

transform = transforms.Compose([
    transforms.ToTensor(),
])

dataset = CustomImageDataset(root_dir=output_dir, transform=transform)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LoRAModel(input_channels=input_channels, hidden_channels=hidden_channels, num_layers=num_layers, kernel_size=kernel_size)
model.to(device)

criterion = nn.MSELoss()
optimizer = Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    total_loss = 0
    for batch in tqdm(dataloader, desc=f"Epoch {epoch+1}/{num_epochs}"):
        images = batch.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, images)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss / len(dataloader)}")

torch.save(model.state_dict(), "lora_model.pth")

# Generate new images using the trained LoRA model
generated_images_dir = "generated_images"
if not os.path.exists(generated_images_dir):
    os.makedirs(generated_images_dir)

eval_dataset = CustomImageDataset(root_dir=output_dir, transform=transform)
eval_dataloader = DataLoader(eval_dataset, batch_size=1, shuffle=False)

for idx, image in enumerate(eval_dataloader):
    image = image.to(device)
    with torch.no_grad():
        generated_image = model(image)
    generated_image = generated_image.clamp(0, 1)
    save_image(generated_image, os.path.join(generated_images_dir, f"generated_image_{idx}.png"))

eval_dataset_generated = CustomImageDataset(root_dir=generated_images_dir, transform=transform)
eval_dataloader_generated = DataLoader(eval_dataset_generated, batch_size=1, shuffle=False)

avg_psnr, avg_ssim = evaluate_model(model, eval_dataloader_generated, device)
print(f"Average PSNR: {avg_psnr}, Average SSIM: {avg_ssim}")


Epoch 1/20: 100%|██████████| 1/1 [00:00<00:00,  4.53it/s]


Epoch 1/20, Loss: 0.3021722435951233


Epoch 2/20: 100%|██████████| 1/1 [00:00<00:00,  5.89it/s]


Epoch 2/20, Loss: 0.2520662546157837


Epoch 3/20: 100%|██████████| 1/1 [00:00<00:00,  5.75it/s]


...

Epoch 19/20, Loss: 0.013902856037020683


Epoch 20/20: 100%|██████████| 1/1 [00:00<00:00,  5.79it/s]


Epoch 20/20, Loss: 0.009683880023658276


Evaluating:   0%|          | 0/11 [00:00<?, ?it/s]<ipython-input-14-fee23a33aac1>:87: FutureWarning: `multichannel` is a deprecated argument name for `structural_similarity`. It will be removed in version 1.0. Please use `channel_axis` instead.
  ssim = SSIM(original_img, generated_img, multichannel=True)
Evaluating: 100%|██████████| 11/11 [00:00<00:00, 49.81it/s]

Average PSNR: 23.51083204282689, Average SSIM: 0.9595467014746233
